In [1]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import simpy

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

#from env.RideSimulator.Grid import Grid

import os,sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from RideSimulator.taxi_sim import run_simulation


In [2]:
#register custom env
import gym

gym.envs.register(
     id='taxi-v0',
     entry_point='env.taxi:TaxiEnv',
     max_episode_steps=1500,
     kwargs={'state_dict':None},
)

In [3]:
#hyper params

num_iterations = 200 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}action

In [4]:
#load taxi env
env_name = "taxi-v0"
env = suite_gym.load(env_name)

#env.reset()
tf_env = tf_py_environment.TFPyEnvironment(env)


In [5]:
#agent and policy
fc_layer_params = (100,)

"""
q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()
"""

#random policy
policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(),
                                                tf_env.action_spec())

"""
#replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffcollect_data_specer_capacity)
    
"""

'\n#replay buffer\nreplay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(\n    data_spec=agent.collect_data_spec,\n    batch_size=train_env.batch_size,\n    max_length=replay_buffcollect_data_specer_capacity)\n    \n'

In [6]:
run_simulation(2)

0 looking for driver at 0
0 got 0 at 0
0 traveling to pickup point, [45 15] from [31 15]
driver 0 location updated  to  [57 23]
Trip distance: 14, time 14
0 finished the trip and released driver 0 at 28 to pool 10
Driver 0 finished trip
1 looking for driver at 200
None
1 looking for driver at 250
1 got 0 at 250
0 traveling to pickup point, [53  0] from [57 23]
driver 0 location updated  to  [25 35]
Trip distance: 45, time 273
2 looking for driver at 300
1 finished the trip and released driver 0 at 318 to pool 5
Driver 0 finished trip
None
2 looking for driver at 350
None
2 looking for driver at 400
2 got 0 at 400
2 looking for driver at 400
2 got 0 at 400
0 traveling to pickup point, [55 24] from [25 35]
driver 0 location updated  to  [19 28]
Trip distance: 36, time 432
2 finished the trip and released driver 0 at 468 to pool 4
Driver 0 finished trip
3 looking for driver at 600
3 got 0 at 600
0 traveling to pickup point, [14 21] from [19 28]
driver 0 location updated  to  [ 4 38]
Trip 

0 traveling to pickup point, [ 2 25] from [27 12]
driver 0 location updated  to  [21 15]
Trip distance: 21, time 8528
48 finished the trip and released driver 0 at 8549 to pool 4
Driver 0 finished trip
49 looking for driver at 8700
None
49 looking for driver at 8750
49 got 0 at 8750
49 looking for driver at 8750
49 got 0 at 8750
49 looking for driver at 8750
49 got 0 at 8750
49 looking for driver at 8750
49 got 0 at 8750
49 looking for driver at 8750
49 got 0 at 8750
49 looking for driver at 8750
49 got 0 at 8750
49 looking for driver at 8750
49 got 0 at 8750
0 traveling to pickup point, [18  9] from [21 15]
driver 0 location updated  to  [59 32]
Trip distance: 47, time 8757
49 finished the trip and released driver 0 at 8804 to pool 11
Driver 0 finished trip
50 looking for driver at 8850
None
50 looking for driver at 8900
None
50 looking for driver at 8950
None
50 looking for driver at 9000
50 got 0 at 9000
0 traveling to pickup point, [28  3] from [59 32]
driver 0 location updated  to

In [7]:
#train

for i in range (num_iterations):
    #run one episode of simulation and record states
    state_dict = run_simulation(policy)
    
    #re-register environemnt with new states
    env_name = 'taxi-v'+int(i)
    gym.envs.register(
         id=env_name,
         entry_point='env.taxi:TaxiEnv',
         max_episode_steps=1500,
         kwargs={'state_dict':state_dict},
    )
    
    #reload new env
    env = suite_gym.load(env_name)
    tf_env = tf_py_environment.TFPyEnvironment(env)

    #reset tf env
    time_step = tf_env.reset()
    
    #loop through recorded steps
    for step in state_dict:
        present_state = tf_env.current_time_step()
        action = step.action
        new_state = tf_env.step(action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)
        replay_buffer.add_batch(traj)
    
print(replay_buffer)

TypeError: handle_trip() takes 5 positional arguments but 6 were given

In [3]:
# startup simulation

def simpy_episode(rewards, steps, time_step, tf_env, policy):

    TIME_MULTIPLIER = 50
    DRIVER_COUNT = 1
    TRIP_COUNT = 8000
    RUN_TIME = 10000
    INTERVAL = 20
    # GRID_WIDTH = 3809
    # GRID_HEIGHT = 2622
    GRID_WIDTH = 60
    GRID_HEIGHT = 40
    HEX_AREA = 2.6

    Env = simpy.Environment()
    map_grid = Grid(env=Env, width=GRID_WIDTH, height=GRID_HEIGHT, interval=INTERVAL, num_drivers=DRIVER_COUNT,
                    hex_area=HEX_AREA)

    taxi_spots = map_grid.taxi_spots
    driver_list = create_drivers(Env, DRIVER_COUNT, map_grid)
    driver_pools = map_grid.driver_pools

    run_simulation(TRIP_COUNT, RUN_TIME, DRIVER_COUNT, TIME_MULTIPLIER, map_grid, taxi_spots, driver_list, driver_pools, Env, rewards, steps, time_step, tf_env, policy)
    t_count = 0
    for dr in driver_list:
        d_t_count = dr.total_trip_count
        t_count += d_t_count
        print(f"{dr.id} completed {d_t_count}")

    print(f"Total trip count: {t_count}")

In [5]:
var = tf.random.uniform([1], 0, 2, dtype=tf.int32)
var[0] = 2
print (var)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [10]:
#simple episode run - atttempt 1

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
    simpy_episode(rewards, step, time_step, tf_env, policy)

    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [ ]:
#simple episode run - atttempt 2

#time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
    time_step = tf_env.reset()
    
    
    
    
    
    
    simpy_episode(rewards, step, time_step, tf_env, policy)

    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

In [ ]:
#simple episode run template
"""
time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)
"""